In [1]:
import re
import numpy as np
import pandas as pd
import pickle
import joblib
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AdamW, get_linear_schedule_with_warmup
from torch.optim import AdamW
from tqdm import tqdm
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, classification_report

# 사용 모델
from sklearn.ensemble import RandomForestClassifier
import lib.EncoderModel as mm

# 함수들 불러오기
import lib.functions as f

# 평가에 이용되는 함수 정의

In [2]:
# 1차 filter용 데이터셋 정의 -> (조문 + 조문제목) [embedding vector화 + 배치화]
class CustomDataset1(Dataset):
    def __init__(self, dataframe, tokenizer):
        self.data = dataframe
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # '조문'과 '법령명' 칼럼의 데이터를 안전하게 문자열로 변환하고 결합
        text1 = str(self.data.iloc[idx]['조문'])
        text2 = str(self.data.iloc[idx]['조문제목'])
        text_data = text1 + " // " + text2
        label = torch.tensor(self.data.iloc[idx]['사무판단'], dtype=torch.long)

        # 결합된 텍스트를 토크나이징
        tokenized_data = tokenizer([text_data], padding='max_length', max_length=512, truncation=True, return_tensors='pt')

        return {
            'input_ids': tokenized_data['input_ids'].squeeze(),
            'attention_mask': tokenized_data['attention_mask'].squeeze(),
            'labels': label
        }

In [3]:
# 2차 filter용 데이터셋 정의 -> (법령명 + 조문제목 + 수행주체) [embedding vector화 + 배치화]
class CustomDataset2(Dataset):
    def __init__(self, dataframe, tokenizer):
        self.data = dataframe
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # '조문'과 '법령명' 칼럼의 데이터를 안전하게 문자열로 변환하고 결합
        text1 = str(self.data.iloc[idx]['법령명'])
        text2 = str(self.data.iloc[idx]['조문제목'])
        text3 = str(self.data.iloc[idx]['수행주체(문자열)'])
        text_data = text1 + " // " + text2 + " // " + text3
        label = torch.tensor(self.data.iloc[idx]['사무유형(대분류)'], dtype=torch.long)

        # 결합된 텍스트를 토크나이징
        tokenized_data = tokenizer([text_data], padding='max_length', max_length=128, truncation=True, return_tensors='pt')

        return {
            'input_ids': tokenized_data['input_ids'].squeeze(),
            'attention_mask': tokenized_data['attention_mask'].squeeze(),
            'labels': label
        }

In [4]:
# 각 배치의 텍스트 길이를 맞추기
def collate_fn(batch):
    input_ids = [item['input_ids'] for item in batch]
    attention_mask = [item['attention_mask'] for item in batch]
    labels = [item['labels'] for item in batch]

    # 패딩 적용
    input_ids = torch.nn.utils.rnn.pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
    attention_mask = torch.nn.utils.rnn.pad_sequence(attention_mask, batch_first=True, padding_value=0)

    return {'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': torch.stack(labels)}


In [5]:
# 평가 결과 확인
def cal_result(preds, y):
    conf_matrix = confusion_matrix(y, preds)
    accuracy = accuracy_score(y, preds)
    precision = precision_score(y, preds)
    recall = recall_score(y, preds)
    f1 = f1_score(y, preds)
    
    # 결과 출력
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1-Score: {f1}")
    print("Confusion Matrix:")
    print(conf_matrix)

In [6]:
def cal_result2(preds, y):
    conf_matrix = confusion_matrix(y, preds)
    print('혼동행렬')
    print(conf_matrix)
    accuracy = accuracy_score(y, preds)
    print(f'정확도: {accuracy}')
    class_report = classification_report(y, preds)
    print('Classification Report:')
    print(class_report)
    
    return conf_matrix

# 1. 데이터 입력

In [7]:
test_df = pd.read_csv('data/val_df.csv')
test_df.info()

/var/folders/zr/_f1rgf8n0w3541q9k8p3smhr0000gn/T/ipykernel_65694/2526232339.py:1: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  test_df = pd.read_csv('data/val_df.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 169615 entries, 0 to 169614
Data columns (total 26 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   소관부처명      169615 non-null  object 
 1   법령명        169615 non-null  object 
 2   법령구분       169615 non-null  int64  
 3   조번호        169502 non-null  object 
 4   항번호        130938 non-null  object 
 5   호번호        99764 non-null   object 
 6   조문제목       169615 non-null  object 
 7   조문         169615 non-null  object 
 8   사무판단       169615 non-null  int64  
 9   사무판단근거     137581 non-null  object 
 10  사무명        10920 non-null   object 
 11  수행주체       10911 non-null   object 
 12  사무유형       10909 non-null   object 
 13  위임사무판단     169615 non-null  int64  
 14  위임근거규정     711 non-null     object 
 15  수임기관       550 non-null     object 
 16  특행기관       169615 non-null  int64  
 17  재위임사무판단    169615 non-null  int64  
 18  재위임근거규정    0 non-null       float64
 19  재수임기관      0 non-null  

# 2. 데이터 전처리

In [8]:
#test_df = f.x_null_drop(test_df)
#test_df = f.x_wrong_drop(test_df)
test_df = f.no_work_check(test_df)
test_df = f.make_large_type(test_df)

In [9]:
# 결과 저장 데이터 프레임 생성
check_cols = ['법령명' ,'조문제목', '조문', '사무판단', '사무유형(대분류)']
sub_df = test_df.loc[:, check_cols].copy()
sub_df.shape

(169615, 5)

# 3. Rule-Base 적용

In [10]:
sub_df = f.rule_based(sub_df)  # rule_base 조문 확인

# rule_base로 걸러낸 조문들
rule_base_filter_idx = sub_df.loc[sub_df['rule_based']==0, :].index

# 4. 1차 사무판단
## encoder 모델

* encoder 구조 생성

In [11]:
# 모델 및 토크나이저 불러오기
model_name = 'klue/roberta-large'
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 모델 기본 구조 설정
input_size = 512
hidden_size = 256
output_size = 2
num_encoder_layers = 1
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = mm.MyModel(input_size, hidden_size, output_size, num_encoder_layers)
model.to(device)

MyModel(
  (encoder): MyEncoder(
    (layers): ModuleList(
      (0): MyEncoderLayer(
        (self_attention): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
        )
        (feedforward): Sequential(
          (0): Linear(in_features=512, out_features=1024, bias=True)
          (1): ReLU()
          (2): Linear(in_features=1024, out_features=512, bias=True)
        )
        (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (layer_norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (layer1): Linear(in_features=512, out_features=256, bias=True)
  (batch_norm): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (layer2): Linear(in_features=256, out_features=2, bias=True)
)

* 학습된 encoder 불러오기

In [12]:
model_path = 'pickles/encoder_model_f1_80.pth'  # Provide the correct path
model.load_state_dict(torch.load(model_path, map_location=device))

<All keys matched successfully>

* 모델 평가

In [13]:
# 데이터로더 생성
batch_size = 16  # 배치크기 조절

test_dataset = CustomDataset1(sub_df, tokenizer)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

In [14]:
model.eval()

test_loss = 0
probs = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc=f'Validation'):
        inputs = {key: value.to(device) for key, value in batch.items() if key != 'labels'}
        labels = batch['labels'].to(device)

        # 모델에 토큰화된 입력 데이터 전달
        outputs = model(**inputs).float()
        test_loss += torch.nn.functional.cross_entropy(outputs, labels.long())
        
        # 소프트맥스를 적용하여 확률값 얻기
        probs_batch = torch.nn.functional.softmax(outputs, dim=1)
        probs.append(probs_batch.cpu().numpy())

# 손실 계산
avg_test_loss = test_loss / len(test_loader)

# 확률값을 하나의 배열로 통합
probs = np.concatenate(probs, axis=0)

print(avg_test_loss)

Validation: 100%|████████████████████████| 10601/10601 [01:02<00:00, 170.26it/s]

tensor(0.3816)


In [15]:
# 결과 확률 저장
prob_1 = probs[:, 1]
prob_1 = [round(prob, 5) for prob in prob_1]

In [16]:
sub_df.head(1)

,법령명,조문제목,조문,사무판단,사무유형(대분류),rule_based
0,환경영향평가법 시행규칙,규제의 재검토,② 환경부장관은 다음 각 호의 사항에 대하여 해당 호에 해당하는 날을 기준으로 2년...,0,0,1


## 1차 사무판단 진행

In [17]:
threshold = 0.251    # test 결과로 확인한 threshold

sub_df['사무판단 예측'] = 0
sub_df['사무판단 예측확률'] = prob_1

# 1차 사무판단
sub_df.loc[sub_df['사무판단 예측확률'] >= threshold, '사무판단 예측'] = 1

sub_df.loc[rule_base_filter_idx, '사무판단 예측'] = 0   # rule-base 결과 적용

In [18]:
#sub_df.loc[(sub_df['사무예측 결과']==0)&(sub_df['사무판단']!=0), :]    # 모델 예측 결과 틀린 부분

In [19]:
# 1차 필터링 결과
cal_result(sub_df['사무판단 예측'], sub_df['사무판단'])

Accuracy: 0.6715679627391445
Precision: 0.15806185692913627
Recall: 0.9491245760381336
F1-Score: 0.2709939148073023
Confusion Matrix:
[[103554  55152]
 [   555  10354]]


## 1차 filtering 결과

In [20]:
# 1차 filter 결과
first_filtered_df = sub_df.loc[sub_df['사무판단 예측']==1, :].copy()
first_filtered_df.head(1)

,법령명,조문제목,조문,사무판단,사무유형(대분류),rule_based,사무판단 예측,사무판단 예측확률
0,환경영향평가법 시행규칙,규제의 재검토,② 환경부장관은 다음 각 호의 사항에 대하여 해당 호에 해당하는 날을 기준으로 2년...,0,0,1,1,0.70103


# 4. 2차 사무판단 + 사무유형 판단
* 사무로 판단되었을때 실행

## Encoder Model

In [21]:
input_size = 128
hidden_size = 64
output_size = 4

total_encoder = mm.MyModel(input_size, hidden_size, output_size)

In [22]:
model_path2 = 'pickles/encoder_model_f2_80.pth'  # Provide the correct path
total_encoder.load_state_dict(torch.load(model_path2, map_location=device))

<All keys matched successfully>

* 수행주체 추출

In [23]:
# 수행주체 종류 불러오기
with open('pickles/subject_list.pkl', 'rb') as file:
    subject_list = pickle.load(file)

In [24]:
# 수행주체(문자열, 리스트 추가)
first_filtered_df = f.new_subject_make(first_filtered_df, subject_list, tqdm)

Processing rows: 100%|██████████████████| 65506/65506 [00:10<00:00, 6088.59it/s]


In [25]:
first_filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 65506 entries, 0 to 169612
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   법령명         65506 non-null  object 
 1   조문제목        65506 non-null  object 
 2   조문          65506 non-null  object 
 3   사무판단        65506 non-null  int64  
 4   사무유형(대분류)   65506 non-null  int64  
 5   rule_based  65506 non-null  int64  
 6   사무판단 예측     65506 non-null  int64  
 7   사무판단 예측확률   65506 non-null  float32
 8   수행주체(리스트)   65506 non-null  object 
 9   수행주체(문자열)   65506 non-null  object 
dtypes: float32(1), int64(4), object(5)
memory usage: 7.3+ MB


In [26]:
# 데이터로더 생성
batch_size = 16  # 배치크기 조절

total_dataset = CustomDataset2(first_filtered_df, tokenizer)
total_loader = DataLoader(total_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

In [27]:
# 2차 인코더 가중치 학습
class_weights = [0.2969,  2.2733,  9.0880, 12.1487]
class_weights = torch.tensor(class_weights, dtype=torch.float32)

In [28]:
total_encoder.eval()
val_loss = 0.0
y_true = []  # 실제 레이블을 저장할 리스트
y_pred = []  # 모델의 예측 결과를 저장할 리스트
y_probs = [] # 사무유형별 확률 저장할 리스트
with torch.no_grad():
    for batch in tqdm(total_loader, desc=f'Test'):
        inputs = {key: value.to(device) for key, value in batch.items() if key != 'labels'}
        labels = batch['labels'].to(device)

        # 모델에 토큰화된 입력 데이터 전달
        outputs = total_encoder(**inputs).float()
        
        probs = torch.nn.functional.softmax(outputs, dim=1)
        y_probs.extend(probs.cpu().numpy())
        _, predictions = torch.max(outputs, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predictions.cpu().numpy())
        val_loss += torch.nn.functional.cross_entropy(outputs, labels, weight=class_weights)

avg_val_loss = val_loss / len(total_loader)
print(f'Test Loss: {avg_val_loss}')

Test: 100%|████████████████████████████████| 4095/4095 [00:15<00:00, 256.46it/s]

Test Loss: 1.280299425125122


In [29]:
probs = pd.DataFrame(y_probs)
probs = probs.set_index(first_filtered_df.index)

# 인코더 결과 저장
encoder_result_df = f.make_result_df(pd, y_true, y_pred, probs[0], probs[1], probs[2], probs[3])

In [30]:
encoder_result_df

,사무유형(대분류),사무유형(대분류) 예측,비사무 확률,국가 확률,지방 확률,공동 확률
0,0,0,0.63346,0.15598,0.10317,0.10739
2,0,0,0.48013,0.20462,0.15528,0.15997
5,0,2,0.26985,0.17593,0.36356,0.19066
6,0,0,0.50363,0.19602,0.12904,0.17131
7,0,0,0.69282,0.16199,0.07278,0.07241
...,...,...,...,...,...,...
169604,0,1,0.28120,0.33264,0.13306,0.25310
169608,0,0,0.68230,0.15938,0.05950,0.09883
169609,0,0,0.32767,0.28323,0.20375,0.18534
169611,0,0,0.35931,0.29082,0.15507,0.19480


In [31]:
# 결과 확인
cal_result2(encoder_result_df['사무유형(대분류) 예측'], encoder_result_df['사무유형(대분류)'])

혼동행렬
[[38621 11633  4153   745]
 [ 3980  2402   706   116]
 [  756   594   409    43]
 [  646   436   220    46]]
정확도: 0.6331939059017495
Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.70      0.78     55152
           1       0.16      0.33      0.22      7204
           2       0.07      0.23      0.11      1802
           3       0.05      0.03      0.04      1348

    accuracy                           0.63     65506
   macro avg       0.29      0.32      0.29     65506
weighted avg       0.76      0.63      0.68     65506



array([[38621, 11633,  4153,   745],
       [ 3980,  2402,   706,   116],
       [  756,   594,   409,    43],
       [  646,   436,   220,    46]])

## Random Forest

In [32]:
rf_2_filtered_df = first_filtered_df.copy()

In [33]:
# RF 모델 불러오기
with open('pickles/rf_model_f2_80.pkl', 'rb') as file:
    model_2_rf = pickle.load(file)

In [34]:
rf_2_filtered_df.head(1)

,법령명,조문제목,조문,사무판단,사무유형(대분류),rule_based,사무판단 예측,사무판단 예측확률,수행주체(리스트),수행주체(문자열)
0,환경영향평가법 시행규칙,규제의 재검토,② 환경부장관은 다음 각 호의 사항에 대하여 해당 호에 해당하는 날을 기준으로 2년...,0,0,1,1,0.70103,"[장관, 환경부]",장관 환경부


In [35]:
# 법령명 통합
rf_2_filtered_df['법령명'] = rf_2_filtered_df['법령명'].apply(lambda x: re.sub(r'\s?시행령|\s?시행규칙', '', x))

In [36]:
# 새로운 컬럼을 이용하기 위한 수행주체 count dictionary
with open('pickles/subject_dic_f2_80.pkl', 'rb') as file:
    subject_dic = pickle.load(file)

In [37]:
# 수행주체 기준 새로운 열 생성
rf_2_filtered_df = f.score_subject(rf_2_filtered_df,subject_dic,tqdm)

Processing rows: 100%|██████████████████| 65506/65506 [00:11<00:00, 5702.20it/s]


In [38]:
select_column = ['법령명', 'score_subject_nan', 'score_subject_n', 'score_subject_r', 'score_subject_p']
rf_2_filtered_label = rf_2_filtered_df['사무유형(대분류)']
rf_2_filtered_input = rf_2_filtered_df[select_column]

In [39]:
# '법령명' label encoding 진행
le = LabelEncoder()
rf_2_filtered_input['법령명'] = le.fit_transform(rf_2_filtered_input['법령명']) # train set에는 fit_transform

/var/folders/zr/_f1rgf8n0w3541q9k8p3smhr0000gn/T/ipykernel_65694/253750670.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rf_2_filtered_input['법령명'] = le.fit_transform(rf_2_filtered_input['법령명']) # train set에는 fit_transform


In [40]:
# rf 결과 추출
y_pred_rf = model_2_rf.predict(rf_2_filtered_input)
y_prob_rf = model_2_rf.predict_proba(rf_2_filtered_input)

In [41]:
# rf 결과 데이터프레임 생성
rf_probs = pd.DataFrame(y_prob_rf)
rf_probs = rf_probs.set_index(first_filtered_df.index)

rf_result_df = f.make_result_df(pd, rf_2_filtered_label, y_pred_rf, rf_probs[0], rf_probs[1], rf_probs[2], rf_probs[3])

In [42]:
# rf 결과 확인
cal_result2(rf_result_df['사무유형(대분류) 예측'], rf_result_df['사무유형(대분류)'])

혼동행렬
[[50505  3373   749   525]
 [ 1978  5167    16    43]
 [  555    16  1222     9]
 [  390    20     4   934]]
정확도: 0.8827893628064605
Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.92      0.93     55152
           1       0.60      0.72      0.65      7204
           2       0.61      0.68      0.64      1802
           3       0.62      0.69      0.65      1348

    accuracy                           0.88     65506
   macro avg       0.69      0.75      0.72     65506
weighted avg       0.89      0.88      0.89     65506



array([[50505,  3373,   749,   525],
       [ 1978,  5167,    16,    43],
       [  555,    16,  1222,     9],
       [  390,    20,     4,   934]])

## Ensemble

In [43]:
pred = [0]*len(y_prob_rf)
e_prob0 = (rf_result_df['비사무 확률'] + encoder_result_df['비사무 확률'])/2
e_prob1 = (rf_result_df['국가 확률'] + encoder_result_df['국가 확률'])/2
e_prob2 = (rf_result_df['지방 확률'] + encoder_result_df['지방 확률'])/2
e_prob3 = (rf_result_df['공동 확률'] + encoder_result_df['공동 확률'])/2

ensemble_result_df_11 = f.make_result_df(pd, rf_2_filtered_label, pred, e_prob0, e_prob1, e_prob2, e_prob3)
ensemble_result_df_11['수행주체(리스트)'] = rf_2_filtered_df['수행주체(리스트)']

In [44]:
# 앙상블 사무유형 판단 저장
ensemble_result_df_11 = f.make_ensemble_pred(ensemble_result_df_11)
ensemble_result_df_11['사무유형(대분류) 예측'] = ensemble_result_df_11['사무유형(대분류) 예측'].astype(int)
ensemble_result_df_11['rule_based'] = first_filtered_df['rule_based']

In [45]:
# 앙살블 결과 확인
cal_result2(ensemble_result_df_11['사무유형(대분류) 예측'], ensemble_result_df_11['사무유형(대분류)'])

혼동행렬
[[51375  2845   536   396]
 [ 2342  4816    10    36]
 [  673    14  1105    10]
 [  470    20     4   854]]
정확도: 0.8877049430586511
Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.93      0.93     55152
           1       0.63      0.67      0.65      7204
           2       0.67      0.61      0.64      1802
           3       0.66      0.63      0.65      1348

    accuracy                           0.89     65506
   macro avg       0.72      0.71      0.72     65506
weighted avg       0.89      0.89      0.89     65506



array([[51375,  2845,   536,   396],
       [ 2342,  4816,    10,    36],
       [  673,    14,  1105,    10],
       [  470,    20,     4,   854]])

In [46]:
# 확실/주의 나누기 [확실: need_to_check = 0 / 주의: need_to_check = 1]
ensemble_result_df_11, good_df, bad_df = f.make_need_to_check(ensemble_result_df_11,
                                                           rf_result_df, encoder_result_df)

In [47]:
ensemble_result_df_11.head(1)

,사무유형(대분류),사무유형(대분류) 예측,비사무 확률,국가 확률,지방 확률,공동 확률,수행주체(리스트),rule_based,need_to_check
0,0,0,0.58967,0.30005,0.05159,0.0587,"[장관, 환경부]",1,0


In [48]:
# 확실 결과
cal_result2(good_df['사무유형(대분류) 예측'].tolist(), good_df['사무유형(대분류)'].tolist())

혼동행렬
[[35634   933    75    15]
 [ 1202  1816     0     0]
 [  271     5   304     0]
 [  210     6     0    37]]
정확도: 0.9329268292682927
Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.97      0.96     36657
           1       0.66      0.60      0.63      3018
           2       0.80      0.52      0.63       580
           3       0.71      0.15      0.24       253

    accuracy                           0.93     40508
   macro avg       0.78      0.56      0.62     40508
weighted avg       0.93      0.93      0.93     40508



array([[35634,   933,    75,    15],
       [ 1202,  1816,     0,     0],
       [  271,     5,   304,     0],
       [  210,     6,     0,    37]])

In [49]:
# 애매 결과
cal_result2(bad_df['사무유형(대분류) 예측'].tolist(), bad_df['사무유형(대분류)'].tolist())

혼동행렬
[[15741  1912   461   381]
 [ 1140  3000    10    36]
 [  402     9   801    10]
 [  260    14     4   817]]
정확도: 0.814425154012321
Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.85      0.87     18495
           1       0.61      0.72      0.66      4186
           2       0.63      0.66      0.64      1222
           3       0.66      0.75      0.70      1095

    accuracy                           0.81     24998
   macro avg       0.70      0.74      0.72     24998
weighted avg       0.83      0.81      0.82     24998



array([[15741,  1912,   461,   381],
       [ 1140,  3000,    10,    36],
       [  402,     9,   801,    10],
       [  260,    14,     4,   817]])

## 2차 filter 결과 저장

In [50]:
sencond_filtered_df = ensemble_result_df_11.copy()   # 2차 필터 결과 저장

In [51]:
# 2차 filter 결과 확인
cal_result2(sencond_filtered_df['사무유형(대분류) 예측'], sencond_filtered_df['사무유형(대분류)'])

혼동행렬
[[51375  2845   536   396]
 [ 2342  4816    10    36]
 [  673    14  1105    10]
 [  470    20     4   854]]
정확도: 0.8877049430586511
Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.93      0.93     55152
           1       0.63      0.67      0.65      7204
           2       0.67      0.61      0.64      1802
           3       0.66      0.63      0.65      1348

    accuracy                           0.89     65506
   macro avg       0.72      0.71      0.72     65506
weighted avg       0.89      0.89      0.89     65506



array([[51375,  2845,   536,   396],
       [ 2342,  4816,    10,    36],
       [  673,    14,  1105,    10],
       [  470,    20,     4,   854]])

In [52]:
# 2차 filter (확실/주의)결과 확인
h_ratio, j_ratio = f.check_result(sencond_filtered_df)

주의 사무 개수: 24998
주의에서 틀린 개수: 4639
주의에서 맞은 개수: 20359
------------------
확실 사무 개수: 40508
확실에서 틀린 개수: 2717
확실에서 맞은 개수: 37791
확실 정확도: 0.9329268292682927
애매 정확도: 0.814425154012321


In [53]:
sencond_filtered_df.head(1)

,사무유형(대분류),사무유형(대분류) 예측,비사무 확률,국가 확률,지방 확률,공동 확률,수행주체(리스트),rule_based,need_to_check
0,0,0,0.58967,0.30005,0.05159,0.0587,"[장관, 환경부]",1,0


In [54]:
sub_df.head(1)

,법령명,조문제목,조문,사무판단,사무유형(대분류),rule_based,사무판단 예측,사무판단 예측확률
0,환경영향평가법 시행규칙,규제의 재검토,② 환경부장관은 다음 각 호의 사항에 대하여 해당 호에 해당하는 날을 기준으로 2년...,0,0,1,1,0.70103


In [55]:
# 최종 결과 저장
sub_df = f.make_final_df(sub_df, sencond_filtered_df)
sub_df

,법령명,조문제목,조문,사무판단,사무유형(대분류),rule_based,사무판단 예측,사무판단 예측확률,수행주체,사무유형(대분류) 결과,비사무 확률,국가 확률,지방 확률,공동 확률,need_to_check
0,환경영향평가법 시행규칙,규제의 재검토,② 환경부장관은 다음 각 호의 사항에 대하여 해당 호에 해당하는 날을 기준으로 2년...,0,0,1.0,1,0.70103,"[장관, 환경부]",0,0.58967,0.30005,0.05159,0.05870,0
1,도서관법,자격취소,2. 제6조제3항에 따라 발급받은 자격증을 다른 사람에게 대여한 경우,0,0,0.0,0,0.01952,,0,0.00000,0.00000,0.00000,0.00000,0
2,장애인복지법 시행규칙,장애인자립생활지원센터의 운영기준,"자립생활센터는 조직 운영, 사업 수행, 재정 확보, 운용 등에 대해 객관적으로 평가...",0,0,1.0,1,0.58506,"[관리, 센터]",0,0.73506,0.10731,0.07764,0.07999,0
3,건설기술 진흥법,건설공사 지원 통합정보체계의 구축,5. 건설공사 지원 통합정보체계를 이용한 정보의 공동활용 촉진,0,0,0.0,0,0.04686,,0,0.00000,0.00000,0.00000,0.00000,0
4,농수산물 유통 및 가격안정에 관한 법률 시행령,도매시장법인의 겸영사업의 제한,2. 제2차 위반: 1개월 금지,0,0,0.0,0,0.01003,,0,0.00000,0.00000,0.00000,0.00000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169610,언론중재 및 피해구제 등에 관한 법률,사망자의 인격권 보호,① 제5조제1항의 타인에는 사망한 사람을 포함한다.,0,0,0.0,0,0.09550,,0,0.00000,0.00000,0.00000,0.00000,0
169611,장애인복지법 시행규칙,시설운영의 중단ㆍ재개ㆍ폐지 신고 등,장애인복지시설을 설치ㆍ운영하는 자는 법 제60조제2항에 따라 시설 운영을 일시중단또...,0,0,1.0,1,0.56394,"[구청, 군수, 구청장, 시장, 청장]",0,0.66301,0.14541,0.09419,0.09740,0
169612,한부모가족지원법,금융정보등의 제공,⑤ 제1항부터 제3항까지의 규정에 따른 금융정보등의 제공 요청과 제공은 「정보통신망...,0,0,1.0,1,0.51660,[],0,0.82633,0.09616,0.02221,0.05531,0
169613,육군종합행정학교령,설치와 임무,①육군에 육군종합행정학교를 둔다. <개정 1999.2.26>,0,0,0.0,0,0.11048,,0,0.00000,0.00000,0.00000,0.00000,0


In [56]:
sub_df.to_csv('data/final_df(val).csv', index=False)